In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../top_rated_wines.csv')
df = df[df["variety"].notna()]
data = df.to_dict('records')

In [3]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [4]:
encoder = SentenceTransformer('all-MiniLM-L6-v2')

In [5]:
qdrant = QdrantClient(":memory:")

qdrant.recreate_collection(
    collection_name="top_wines",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),
        distance=models.Distance.COSINE
    )
)

/tmp/ipykernel_18552/1510119624.py:3: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [6]:
qdrant.upload_points(
    collection_name="top_wines",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["notes"]).tolist(),
            payload=doc
        ) for idx, doc in enumerate(data)
    ]
)

In [7]:
user_prompt = "Suggest me an amazing Malbec wine from Argentina"

In [8]:
hits = qdrant.search(
    collection_name="top_wines",
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=3
)

search_results = [hit.payload for hit in hits]

In [9]:
search_results

[{'name': 'Catena Zapata Argentino Vineyard Malbec 2004',
  'region': 'Argentina',
  'variety': 'Red Wine',
  'rating': 98.0,
  'notes': '"The single-vineyard 2004 Malbec Argentino Vineyard spent 17 months in new French oak. Remarkably fragrant and complex aromatically, it offers up aromas of wood smoke, creosote, pepper, clove, black cherry, and blackberry. Made in a similar, elegant style, it is the most structured of the three single vineyard wines, needing a minimum of a decade of additional cellaring. It should easily prove to be a 25-40 year wine. It is an exceptional achievement in Malbec. When all is said and done, Catena Zapata is the Argentina winery of reference – the standard of excellence for comparing all others. The brilliant, forward-thinking Nicolas Catena remains in charge, with his daughter, Laura, playing an increasingly large role. The Catena Zapata winery is an essential destination for fans of both architecture and wine in Mendoza. It is hard to believe, given th

How to run model: [Quickstart](https://github.com/Mozilla-Ocho/llamafile?tab=readme-ov-file#quickstart)

In [10]:
from openai import OpenAI
client = OpenAI(
    base_url="http://127.0.0.1:8080/v1",
    api_key="sk-no-key-required"
)

completion = client.chat.completions.create(
    model="LLaMa_CPP",
    messages=[
        {"role": "system", "content": "You are a chatbot, a wine specialist. Your top priority is to help guide users into selecting amazing wine and guide them with their requests."},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": str(search_results)}
    ]
)

print(completion.choices[0].message)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ChatCompletionMessage(content='[{\'name\': \'Catena Zapata Adrianna Vineyard Malbec 2004\', \'region\': \'Argentina\', \'variety\': \'Red Wine\', \'rating\': 97.0, \'notes\': \'"The single-vineyard 2004 Malbec Adrianna Vineyard from the Gualtallary district is inky purple with aromas of wood smoke, pencil lead, game, black cherry, and blackberry liqueur. Opulent, full-flavored, yet remarkably light on its feet, this medium to full-bodied Malbec is all about pleasure. It will certainly evolve for a decade but is hard to resist now. It is a fine test of one\'s ability to defer immediate gratification. When all is said and done, Catena Zapata is the Argentina winery of reference – the standard of excellence for comparing all others. The brilliant, forward-thinking Nicolas Catena remains in charge, with his daughter, Laura, playing an increasingly large role. The Catena Zapata winery is an essential destination for fans of both architecture and wine in Mendoza. It is hard to believe, given

In [40]:
print(completion.choices[0].message.content)

[{'name': 'Catena Zapata Adrianna Vineyard Malbec 2004', 'region': 'Argentina', 'variety': 'Red Wine', 'rating': 97.0, 'notes': '"The single-vineyard 2004 Malbec Adrianna Vineyard from the Gualtallary district is inky purple with aromas of wood smoke, pencil lead, game, black cherry, and blackberry liqueur. Opulent, full-flavored, yet remarkably light on its feet, this medium to full-bodied Malbec is all about pleasure. It will certainly evolve for a decade but is hard to resist now. It is a fine test of one's ability to defer immediate gratification. When all is said and done, Catena Zapata is the Argentina winery of reference – the standard of excellence for comparing all others. The brilliant, forward-thinking Nicolas Catena remains in charge, with his daughter, Laura, playing an increasingly large role. The Catena Zapata winery is an essential destination for fans of both architecture and wine in Mendoza. It is hard to believe, given the surge in popularity of Malbec in recent year